In [1]:
import os
import sys
import torch
import torchaudio
import cv2
import numpy as np
import time
import tempfile
import moviepy.editor as mp
from collections import deque
from threading import Thread, Lock
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                            QHBoxLayout, QLabel, QPushButton, QSlider, QStyle, 
                            QFileDialog, QFrame)
from PyQt5.QtCore import Qt, QTimer, pyqtSignal, QThread, QMutex
from PyQt5.QtGui import QImage, QPixmap, QPainter, QColor, QPen, QBrush

# Paths and constants
SAVED_MODEL_PATH = 'F:/SRC_Bhuvaneswari/typpo/Crimenet/W2V/Checkpoint/wav2vec2_epoch_10.pt'
LABEL_MAP = {0: 'Normal', 1: 'Abuse', 2: 'Explosion', 3: 'Fighting', 4: 'Car Accident', 5: 'Shooting', 6: 'Riot'}
COLOR_MAP = {
    'Normal': QColor(50, 200, 50),     # Green
    'Abuse': QColor(255, 0, 0),        # Bright Red
    'Explosion': QColor(255, 127, 0),  # Orange
    'Fighting': QColor(200, 50, 50),   # Red
    'Car Accident': QColor(50, 50, 200), # Blue
    'Shooting': QColor(200, 0, 200),   # Purple
    'Riot': QColor(255, 255, 0)        # Yellow
}
SAMPLING_RATE = 16000
WINDOW_SIZE_SECONDS = 3  # Process 3 seconds of audio at a time

# Load model and processor
print("Loading Wav2Vec2 model and processor...")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=len(LABEL_MAP)
)
model.load_state_dict(torch.load(SAVED_MODEL_PATH))
model.eval()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on {device}")

Loading Wav2Vec2 model and processor...


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on cuda


In [2]:
# Class for audio prediction worker thread
class AudioPredictionWorker(QThread):
    predictionReady = pyqtSignal(float, float, str)
    
    def __init__(self, model, processor, sampling_rate):
        super().__init__()
        self.model = model
        self.processor = processor
        self.sampling_rate = sampling_rate
        self.audio_queue = deque()
        self.running = True
        self.mutex = QMutex()
        
    def add_audio_segment(self, audio_segment, start_time, end_time):
        self.mutex.lock()
        self.audio_queue.append((audio_segment, start_time, end_time))
        self.mutex.unlock()
        
    def stop(self):
        self.running = False
        
    def run(self):
        while self.running:
            if self.audio_queue:
                self.mutex.lock()
                audio_segment, start_time, end_time = self.audio_queue.popleft()
                self.mutex.unlock()
                
                # Process and make prediction
                try:
                    # Process the audio
                    inputs = self.processor(audio_segment, sampling_rate=self.sampling_rate, 
                                           return_tensors="pt", padding=True)
                    inputs = {k: v.to(device) for k, v in inputs.items()}
                    
                    # Make prediction
                    with torch.no_grad():
                        outputs = self.model(**inputs)
                        logits = outputs.logits
                        predicted_id = torch.argmax(logits, dim=-1).item()
                        predicted_class = LABEL_MAP[predicted_id]
                    
                    # Emit the prediction result
                    self.predictionReady.emit(start_time, end_time, predicted_class)
                    
                except Exception as e:
                    print(f"Error making prediction: {e}")
            
            # Sleep to avoid high CPU usage
            time.sleep(0.1)

In [3]:
# Audio extraction class
class AudioExtractor:
    def __init__(self, video_path, chunk_duration=WINDOW_SIZE_SECONDS):
        self.video_path = video_path
        self.chunk_duration = chunk_duration
        self.sampling_rate = SAMPLING_RATE
        
        # Extract full audio
        self.temp_dir = tempfile.mkdtemp()
        self.temp_audio_path = os.path.join(self.temp_dir, "temp_audio.wav")
        video = mp.VideoFileClip(video_path)
        self.duration = video.duration
        video.audio.write_audiofile(self.temp_audio_path, fps=self.sampling_rate, verbose=False, logger=None)
        
        # Load audio
        self.waveform, self.orig_sample_rate = torchaudio.load(self.temp_audio_path)
        self.waveform = self.waveform.mean(dim=0)  # Convert to mono
        
    def get_audio_segment(self, start_time, end_time):
        # Convert time to samples
        start_sample = int(start_time * self.sampling_rate)
        end_sample = int(end_time * self.sampling_rate)
        
        # Ensure within bounds
        if start_sample >= self.waveform.size(0) or end_sample <= 0:
            return None
        
        start_sample = max(0, start_sample)
        end_sample = min(self.waveform.size(0), end_sample)
        
        # Extract segment
        segment = self.waveform[start_sample:end_sample].numpy()
        return segment
    
    def cleanup(self):
        # Clean up temporary files
        try:
            os.remove(self.temp_audio_path)
            os.rmdir(self.temp_dir)
        except:
            pass

In [4]:
# Main application window
class AudioVideoPlayerWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        
        # Setup UI
        self.setWindowTitle("Audio Anomaly Detection Video Player")
        self.setGeometry(100, 100, 1000, 600)
        
        # Create central widget and layout
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)
        
        # Video display area
        self.video_frame = QLabel()
        self.video_frame.setAlignment(Qt.AlignCenter)
        self.video_frame.setMinimumSize(640, 480)
        self.video_frame.setStyleSheet("background-color: black;")
        self.layout.addWidget(self.video_frame)
        
        # Current classification display
        self.classification_label = QLabel("Classification: None")
        self.classification_label.setAlignment(Qt.AlignCenter)
        self.classification_label.setStyleSheet("font-size: 18px; font-weight: bold;")
        self.layout.addWidget(self.classification_label)
        
        # Timeline widget
        self.timeline_widget = TimelineWidget()
        self.layout.addWidget(self.timeline_widget)
        
        # Controls layout
        self.controls_layout = QHBoxLayout()
        
        # Play/Pause button
        self.play_button = QPushButton()
        self.play_button.setIcon(self.style().standardIcon(QStyle.SP_MediaPlay))
        self.play_button.clicked.connect(self.toggle_play)
        self.controls_layout.addWidget(self.play_button)
        
        # Time display
        self.time_label = QLabel("00:00 / 00:00")
        self.controls_layout.addWidget(self.time_label)
        
        # Position slider
        self.position_slider = QSlider(Qt.Horizontal)
        self.position_slider.sliderMoved.connect(self.set_position)
        self.controls_layout.addWidget(self.position_slider)
        
        # Open file button
        self.open_button = QPushButton("Open Video")
        self.open_button.clicked.connect(self.open_file)
        self.controls_layout.addWidget(self.open_button)
        
        self.layout.addLayout(self.controls_layout)
        
        # Video processing variables
        self.cap = None
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.current_frame = 0
        self.fps = 0
        self.total_frames = 0
        self.playing = False
        
        # Audio processing variables
        self.audio_extractor = None
        self.anomaly_segments = []
        self.current_time = 0
        self.last_prediction_time = 0
        
        # Start prediction thread
        self.prediction_worker = AudioPredictionWorker(model, processor, SAMPLING_RATE)
        self.prediction_worker.predictionReady.connect(self.update_anomaly)
        self.prediction_worker.start()

    def open_file(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Open Video File", "", 
                                                "Video Files (*.mp4 *.avi *.mkv *.mov);;All Files (*)")
        if file_path:
            self.load_video(file_path)
    
    def load_video(self, file_path):
        # Stop current video if playing
        if self.playing:
            self.timer.stop()
            self.playing = False
        
        # Release previous capture if any
        if self.cap is not None:
            self.cap.release()
        
        # Cleanup previous audio extractor
        if self.audio_extractor:
            self.audio_extractor.cleanup()
        
        # Open new video
        self.cap = cv2.VideoCapture(file_path)
        if not self.cap.isOpened():
            print(f"Error: Could not open video {file_path}")
            return
        
        # Extract audio
        print("Extracting audio from video...")
        self.audio_extractor = AudioExtractor(file_path)
        
        # Get video properties
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.current_frame = 0
        self.current_time = 0
        self.last_prediction_time = 0
        
        # Clear previous anomalies
        self.anomaly_segments = []
        self.timeline_widget.set_anomalies([])
        self.timeline_widget.set_duration(self.audio_extractor.duration)
        
        # Reset UI
        self.position_slider.setRange(0, self.total_frames)
        duration_str = self.format_time(self.total_frames / self.fps)
        self.time_label.setText(f"00:00 / {duration_str}")
        self.classification_label.setText("Classification: None")
        
        # Show first frame
        ret, frame = self.cap.read()
        if ret:
            self.display_frame(frame)
        
        print("Video loaded successfully")
    
    def toggle_play(self):
        if self.cap is None:
            return
            
        if self.playing:
            self.timer.stop()
            self.play_button.setIcon(self.style().standardIcon(QStyle.SP_MediaPlay))
        else:
            self.timer.start(1000 // 30)  # 30 fps display
            self.play_button.setIcon(self.style().standardIcon(QStyle.SP_MediaPause))
        
        self.playing = not self.playing
    
    def update_frame(self):
        if self.cap is None or not self.playing:
            return
            
        ret, frame = self.cap.read()
        if not ret:
            # End of video
            self.timer.stop()
            self.playing = False
            self.play_button.setIcon(self.style().standardIcon(QStyle.SP_MediaPlay))
            return
        
        # Display the frame
        self.display_frame(frame)
        
        # Update current time
        self.current_time = self.current_frame / self.fps
        
        # Process audio segment for prediction
        if self.current_time - self.last_prediction_time >= WINDOW_SIZE_SECONDS / 2:  # 50% overlap
            start_time = max(0, self.current_time - WINDOW_SIZE_SECONDS / 2)
            end_time = start_time + WINDOW_SIZE_SECONDS
            
            # Get audio segment
            audio_segment = self.audio_extractor.get_audio_segment(start_time, end_time)
            if audio_segment is not None:
                # Add to prediction queue
                self.prediction_worker.add_audio_segment(audio_segment, start_time, end_time)
            
            self.last_prediction_time = self.current_time
        
        # Update slider and time label
        self.position_slider.setValue(self.current_frame)
        current_time_str = self.format_time(self.current_time)
        duration_str = self.format_time(self.total_frames / self.fps)
        self.time_label.setText(f"{current_time_str} / {duration_str}")
        
        # Update classification display
        current_class = self.get_classification_at_time(self.current_time)
        self.classification_label.setText(f"Classification: {current_class}")
        
        # Set classification color
        color = COLOR_MAP.get(current_class, QColor(100, 100, 100))
        self.classification_label.setStyleSheet(f"font-size: 18px; font-weight: bold; color: rgb({color.red()}, {color.green()}, {color.blue()})")
        
        # Increment frame counter
        self.current_frame += 1
    
    def display_frame(self, frame):
        # Convert frame to QImage and display
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame_rgb.shape
        bytes_per_line = ch * w
        q_img = QImage(frame_rgb.data, w, h, bytes_per_line, QImage.Format_RGB888)
        self.video_frame.setPixmap(QPixmap.fromImage(q_img).scaled(
            self.video_frame.width(), self.video_frame.height(), 
            Qt.KeepAspectRatio, Qt.SmoothTransformation))
    
    def set_position(self, position):
        if self.cap is None:
            return
            
        # Seek to position
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, position)
        self.current_frame = position
        self.current_time = position / self.fps
        
        # Read and display the frame
        ret, frame = self.cap.read()
        if ret:
            self.display_frame(frame)
            
            # Update time label
            current_time_str = self.format_time(self.current_time)
            duration_str = self.format_time(self.total_frames / self.fps)
            self.time_label.setText(f"{current_time_str} / {duration_str}")
            
            # Update classification display
            current_class = self.get_classification_at_time(self.current_time)
            self.classification_label.setText(f"Classification: {current_class}")
            
            # Set classification color
            color = COLOR_MAP.get(current_class, QColor(100, 100, 100))
            self.classification_label.setStyleSheet(f"font-size: 18px; font-weight: bold; color: rgb({color.red()}, {color.green()}, {color.blue()})")
    
    def update_anomaly(self, start_time, end_time, anomaly_type):
        # Add new anomaly segment
        self.anomaly_segments.append((start_time, end_time, anomaly_type))
        # Update timeline display
        self.timeline_widget.set_anomalies(self.anomaly_segments)
    
    def get_classification_at_time(self, time_point):
        # Find the most recent classification for the current time
        for start_time, end_time, anomaly_type in reversed(self.anomaly_segments):
            if start_time <= time_point and time_point <= end_time:
                return anomaly_type
        return "None"
    
    def format_time(self, seconds):
        minutes, seconds = divmod(int(seconds), 60)
        hours, minutes = divmod(minutes, 60)
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}" if hours else f"{minutes:02d}:{seconds:02d}"
    
    def closeEvent(self, event):
        # Clean up resources
        if self.cap is not None:
            self.cap.release()
        if self.audio_extractor:
            self.audio_extractor.cleanup()
        self.prediction_worker.stop()
        self.prediction_worker.wait()
        event.accept()

In [5]:
# Timeline widget to show anomaly segments
class TimelineWidget(QFrame):
    def __init__(self):
        super().__init__()
        self.setMinimumHeight(40)
        self.setStyleSheet("background-color: #2d2d2d;")
        
        self.anomalies = []
        self.duration = 0
        
    def set_anomalies(self, anomalies):
        self.anomalies = anomalies
        self.update()
        
    def set_duration(self, duration):
        self.duration = duration
        self.update()
        
    def paintEvent(self, event):
        if self.duration <= 0:
            return
            
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        # Draw background
        painter.fillRect(self.rect(), QBrush(QColor(45, 45, 45)))
        
        # Draw timeline base
        painter.setPen(QPen(QColor(200, 200, 200), 1))
        y_middle = self.height() // 2
        painter.drawLine(0, y_middle, self.width(), y_middle)
        
        # Draw time markers
        painter.setPen(QPen(QColor(150, 150, 150), 1))
        marker_interval = self.width() / 10
        for i in range(11):
            x = i * marker_interval
            painter.drawLine(int(x), y_middle - 5, int(x), y_middle + 5)
            
            # Draw time text
            time_at_marker = (i / 10) * self.duration
            minutes = int(time_at_marker / 60)
            seconds = int(time_at_marker % 60)
            time_text = f"{minutes:02d}:{seconds:02d}"
            painter.drawText(int(x) - 15, y_middle + 20, time_text)
        
        # Draw anomaly segments
        for start_time, end_time, anomaly_type in self.anomalies:
            if start_time >= self.duration:
                continue
                
            # Calculate positions
            start_pos = int((start_time / self.duration) * self.width())
            end_pos = int((min(end_time, self.duration) / self.duration) * self.width())
            
            # Get color for anomaly type
            color = COLOR_MAP.get(anomaly_type, QColor(100, 100, 100))
            
            # Draw segment
            painter.fillRect(start_pos, 5, end_pos - start_pos, self.height() - 10, QBrush(color))
            
            # Draw label if segment is wide enough
            if end_pos - start_pos > 50:
                painter.setPen(QPen(QColor(255, 255, 255), 1))
                painter.drawText(start_pos + 5, y_middle + 5, anomaly_type)

In [6]:
# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = AudioVideoPlayerWindow()
    window.show()
    sys.exit(app.exec_())

Extracting audio from video...
Video loaded successfully


SystemExit: 0